In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Hyperparams
BATCH_SIZE = 128
LR = 0.0005
NUM_WORKERS = 8
EPOCHS=100
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4

In [3]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda:1')
    else:
        return torch.device('cpu')
DEVICE = get_default_device()

In [4]:
normMean = [0.485, 0.456, 0.406]
normStd = [0.229, 0.224, 0.225]
normTransform = transforms.Normalize(normMean, normStd)

trainTransform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(10),
        transforms.RandomCrop(224, padding=16),
        transforms.ToTensor(),
        normTransform
    ])
testTransform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        normTransform
    ])

In [5]:
train_dataset = datasets.CIFAR10(root='data', train=True, download=True, transform=trainTransform)
test_dataset = datasets.CIFAR10(root='data', train=False, download=True, transform=testTransform)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = torchvision.models.resnet50(pretrained=True)
    # for p in model.parameters():
    #     p.requires_grad = False
num_fltrs = model.fc.in_features
model.fc = nn.Sequential(
            nn.Linear(num_fltrs , 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.GELU(),
            nn.Linear(512 , 256),
            nn.BatchNorm1d(256),
            nn.Dropout(0.3),
            nn.GELU(),
            nn.Linear(256 , 10),
            nn.Softmax(dim=1))

model = model.to(DEVICE)

In [8]:
# import torch
# import torch.nn as nn
# inp = torch.tensor([1.0, 2.0, 3, 4, 5])

# outplace_dropout = nn.Dropout(p=0.5)
# print(inp)
# output = outplace_dropout(inp)
# print(output)
# print(inp) # Notice that the input doesn't get changed here


# inplace_droput = nn.Dropout(p=0.4, inplace=True)
# inplace_droput(inp)
# print(inp) # Notice that the input is changed now

In [9]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), LR,
                            momentum=MOMENTUM,
                            weight_decay=WEIGHT_DECAY, nesterov=True)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0005, max_lr=0.001)

In [11]:
def trainEpoch(device, model, loader, optimizer, criterion):
    grad_clip = 0.1
    loop = tqdm(loader)
    model.train()

    losses = []
    correct = 0
    total = 0
    for batch_id, (images, labels) in enumerate(loop):
        images, labels = images.to(device), labels.to(device)
        model = model.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()

        loss = criterion(outputs, labels)
        losses.append(loss.item())


        optimizer.zero_grad()
        loss.backward()
        if grad_clip: 
            nn.utils.clip_grad_value_(model.parameters(), grad_clip)
        
        optimizer.step()
        loop.set_postfix(train_loss=loss.item(), train_accuracy=(100*correct/total))

    return np.mean(losses), 100*correct/total


def testEpoch(device, model, loader, criterion):
    
    model.eval()
    with torch.no_grad():
        losses = []
        correct = 0
        total = 0
        loop = tqdm(loader)
        for batch_id, (images, labels) in enumerate(loop):
            images, labels = images.to(device), labels.to(device)
            model = model.to(device)
        
            outputs = model(images)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()

            loss = criterion(outputs, labels)
            losses.append(loss.item())

            loop.set_postfix(test_loss=loss.item(), test_accuracy=(100*correct/total))

        return np.mean(losses), 100*correct/total

In [12]:
last_best = 0
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for epoch in range (EPOCHS):
    
    train_loss, train_accuracy = trainEpoch(DEVICE, model, train_loader, optimizer, criterion)
    test_loss, test_accuracy = testEpoch(DEVICE, model, test_loader, criterion)

    train_losses.append(train_loss)
    train_acc.append(train_accuracy)
    test_losses.append(test_loss)
    test_acc.append(test_accuracy)

    scheduler.step()

    if last_best <= test_accuracy:
        torch.save(model.state_dict(), 'resnet_50_pretrained_baseline.pt')
        print("Saving new Best Model!")
        last_best = test_accuracy

    print("Train losses: {0:.3f},   Train acc: {1:.3f} ".format(train_losses[-1], train_acc[-1]))
    print("Test losses: {0:.3f},   Test acc: {1:.3f} ".format(test_losses[-1], test_acc[-1]))

dict = {'train_loss': train_losses, 'train_accuracy': train_acc, 'test_loss': test_losses, 'test_accuracy': test_acc}
df = pd.DataFrame(dict)
df.to_csv('loss_and_acc_resnet50_baseline.csv')

100%|███████| 79/79 [00:12<00:00,  6.38it/s, test_accuracy=62.2, test_loss=1.98]


Saving new Best Model!
Train losses: 2.226,   Train acc: 31.508 
Test losses: 2.073,   Test acc: 62.180 


100%|███████| 79/79 [00:12<00:00,  6.38it/s, test_accuracy=83.7, test_loss=1.62]


Saving new Best Model!
Train losses: 1.933,   Train acc: 71.006 
Test losses: 1.758,   Test acc: 83.740 


100%|████████| 79/79 [00:12<00:00,  6.31it/s, test_accuracy=90.3, test_loss=1.6]


...

Train losses: 1.465,   Train acc: 99.696 
Test losses: 1.503,   Test acc: 96.080 


100%|█████████| 79/79 [00:12<00:00,  6.47it/s, test_accuracy=96, test_loss=1.47]


Train losses: 1.466,   Train acc: 99.696 
Test losses: 1.501,   Test acc: 96.040 


100%|███████| 79/79 [00:12<00:00,  6.45it/s, test_accuracy=96.1, test_loss=1.46]

Train losses: 1.465,   Train acc: 99.724 
Test losses: 1.501,   Test acc: 96.090 
